In [3]:
#####Symmetric Correspondence LDA#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [4]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [60]:
####データの発生####
##データの設定
L = 3   #データ数
k = 15   #トピック数
d = 3000   #文書数
v1 = 1000; v2 = 800; v3 = 700   
v = [v1, v2, v3]   #語彙数
w1 = np.random.poisson(np.random.gamma(85, 1/0.6, d), d)
w2 = np.random.poisson(np.random.gamma(80, 1/0.65, d), d)
w3 = np.random.poisson(np.random.gamma(75, 1/0.65, d), d)
w = [w1, w2, w3]   #単語数
f1 = np.sum(w1); f2 = np.sum(w2); f3 = np.sum(w3)   #総単語数
f = np.array([f1, f2, f3])

In [93]:
##IDとインデックスの設定
#IDの設定
d_id1 = np.repeat(range(d), w1)
d_id2 = np.repeat(range(d), w2)
d_id3 = np.repeat(range(d), w3)

#インデックスの設定
d_list1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]
d_list3 = [i for i in range(d)]
for i in range(d):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")
    d_list3[i] = np.array(np.where(d_id3==i)[0], dtype="int")

In [94]:
##データの生成
rp = 0
while True:
    rp = rp + 1
    print(rp)

    ##パラメータの生成
    #ディリクレ分布のパラメータ
    alpha1 = np.repeat(0.4, L)
    alpha2 = np.repeat(0.15, k)
    beta1 = np.repeat(0.05, v1)
    beta2 = np.repeat(0.05, v2)
    beta3 = np.repeat(0.05, v3)

    #ディリクレ分布からトピック分布を生成
    theta1 = np.random.dirichlet(alpha1, d)
    theta2 = np.zeros((d, k, L))
    for j in range(L):
        theta2[:, :, j] = np.random.dirichlet(alpha2, d)
    thetat1 = theta1.copy(); thetat2 = theta2.copy()

    #ディリクレ分布から単語分布を生成
    phi1 = np.random.dirichlet(beta1, k)  
    phi2 = np.random.dirichlet(beta2, k)  
    phi3 = np.random.dirichlet(beta3, k)  

    #出現確率が低い単語を入れ替える
    index_v1 = np.array(range(v1))[np.max(phi1, axis=0) <= (k*5)/f1]
    for j in range(index_v1.shape[0]):
        phi1[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v1[j]] = (k*5)/f1
    phi1 = phi1 / np.sum(phi1, axis=1).reshape(k, 1)
    phit1 = phi1.copy()

    index_v2 = np.array(range(v2))[np.max(phi2, axis=0) <= (k*5)/f2]
    for j in range(index_v2.shape[0]):
        phi2[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v2[j]] = (k*5)/f1
    phi2 = phi2 / np.sum(phi2, axis=1).reshape(k, 1)
    phit2 = phi2.copy()

    index_v3 = np.array(range(v3))[np.max(phi3, axis=0) <= (k*5)/f3]
    for j in range(index_v3.shape[0]):
        phi3[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v3[j]] = (k*5)/f3
    phi3 = phi3 / np.sum(phi3, axis=1).reshape(k, 1)
    phit3 = phi3.copy()

    #単語分布をリストに格納
    phi = [phi1, phi2, phi3]
    phit = phi.copy()

    
    ##トピックと単語を生成
    #モードごとのオブジェクトの格納用リスト
    Z1_list = [j for j in range(L)]
    Z2_list = [j for j in range(L)]
    WX_list = [j for j in range(L)]
    word_list = [j for j in range(L)]

    for m in range(L):
        #文書ごとのオブジェクトの格納用リスト
        WX = np.zeros((d, v[m]), dtype="int32")
        word_data = [i for i in range(d)]
        Z1 = [i for i in range(d)]
        Z2 = [i for i in range(d)]

        for i in range(d):
            #モード変数を生成
            n = w[m][i]
            z1 = np.random.multinomial(1, theta1[i, ], n)
            z1_vec = np.dot(z1, np.arange(L))

            #トピックを生成
            z2 = np.array(rmnom(theta2[i, :, z1_vec], n, k, np.arange(n)).todense())
            z2_vec = np.dot(z2, np.arange(k))

            #単語の生成
            words = np.array(rmnom(phi[m][z2_vec, ], n, v[m], np.arange(n)).todense())
            word_vec = np.dot(words, np.arange(v[m]))

            #文書ごとにデータを格納
            Z1[i] = z1
            Z2[i] = z2
            WX[i, ] = np.sum(words, axis=0)
            word_data[i] = words

        #オブジェクトごとにデータを格納
        WX_list[m] = WX
        word_list[m] = np.array(list(itertools.chain(*[word_data[i] for i in range(d)])))
        Z1_list[m] = np.array(list(itertools.chain(*[Z1[i] for i in range(d)])))
        Z2_list[m] = np.array(list(itertools.chain(*[Z2[i] for i in range(d)])))
    
    #単語がすべて出現していたらbreak
    min_word = np.zeros((L), dtype="int32")
    for j in range(L):
        min_word[j] = np.min(np.sum(WX_list[j], axis=0))

    if np.min(min_word) > 0:
        print(min_word)
        break

1
2
[2 2 2]


In [103]:
#リストを変換
Z11 = Z1_list[0]; Z12 = Z1_list[1]; Z13 = Z1_list[2]
Z21 = Z2_list[0]; Z22 = Z2_list[1]; Z23 = Z2_list[2]
wd1 = np.dot(word_list[0], np.arange(v1))
wd2 = np.dot(word_list[1], np.arange(v2))
wd3 = np.dot(word_list[2], np.arange(v3))
word_data1 = sparse.csr_matrix(word_list[0])
word_data2 = sparse.csr_matrix(word_list[1])
word_data3 = sparse.csr_matrix(word_list[2])
del Z1_list; del Z2_list; del word_list; del WX